## Q1: How many convolutional layers are in the ResNet-18 model ?

Ans - 17

## Q2: What is the input dimension of the last linear layer ?

Ans - 512

## Q3: How many trainable parameters and how many gradients in the ResNet-18 model that you build when using SGD

In [1]:
from resnet_test import *
import torch
from torch.optim import SGD,  Adam
import torch.nn.functional as F



def get_params(lis):
    return sum(p.numel() for p in lis if p.requires_grad)

def get_gradients(lis):
    return sum(p.grad.numel() for p in lis if p.requires_grad)

To get the number of  gradients, backprop needs to be done atleast once

In [4]:
net = ResNet18()

inp = torch.ones(64, 3, 32, 32)
labels = torch.ones(64).long()

opt_sgd = SGD(net.parameters(), lr = 0.01)

out = net(inp)
loss = F.cross_entropy(out, labels)

opt_sgd.zero_grad()
loss.backward()
opt_sgd.step()
print('Number of trainable parameters are',get_params(opt_sgd.param_groups[0]['params']))
print('Number of gradients are',get_gradients(opt_sgd.param_groups[0]['params']))

Number of trainable parameters are 11173962
Number of gradients are 11173962


## Q4: Same question as Q2, except now using Adam

input dimension of the last linear layer is still 512